In [1]:
import sys
sys.path.append('a/UltimeTradingBot/Crypto_backtest_tools')
from utilities.get_data import get_historical_from_db
from utilities.backtesting import basic_single_asset_backtest, plot_wallet_vs_asset, get_metrics
import pandas as pd
import ccxt
import matplotlib.pyplot as plt
import ta
import numpy as np
import random
import seaborn as sns
from utilities.backtesting import plot_wallet_vs_asset, get_metrics, get_n_columns, basic_multi_asset_backtest, plot_sharpe_evolution, plot_bar_by_month
from utilities.custom_indicators import SuperTrend
pd.options.mode.chained_assignment = None  # default='warn'


In [2]:

import tensorflow as tf
# tf.test.is_gpu_available(
#     cuda_only=False, min_cuda_compute_capability=None
# )
tf.config.list_physical_devices('GPU')



[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
PRERR=False
def prerr(err):
    if PRERR:
        print("\033[0;31m Error in "+str(sys._getframe().f_code.co_name) +" \033[0;33m"+str(err))

In [4]:
Binance_USDT_HALAL = [
    "BTC/USDT",
    "LUNA/USDT",
    "ETH/USDT",
    "GMT/USDT",
    "UST/USDT",
    "SOL/USDT",
    "APE/USDT",
    "XRP/USDT",
    "IDEX/USDT",
    "AVAX/USDT",
    "DOT/USDT",
    "ADA/USDT",
    "JASMY/USDT",
    "TRX/USDT",
    "NEAR/USDT",
    "AXS/USDT",
    "GAL/USDT",
    "GALA/USDT",
    "SHIB/USDT",
    "ZIL/USDT",
    "ENS/USDT",
    "DOGE/USDT",
    "LTC/USDT",
    "EUR/USDT",
    "MANA/USDT",
    "DAR/USDT",
    "WAVES/USDT",
    "LAZIO/USDT",
    "ALICE/USDT",
    "ROSE/USDT",
    "ZEC/USDT",
    "ALGO/USDT",
    "GRT/USDT",
    "PSG/USDT",
    "SLP/USDT",
    "EOS/USDT",
    "PORTO/USDT",
    "ICP/USDT",
    "EGLD/USDT",
    "XMR/USDT",
    "KDA/USDT",
    "ETC/USDT",
    "MBOX/USDT",
    "OGN/USDT",
    "AR/USDT",
    "GLMR/USDT",
    "LOKA/USDT",
    "XLM/USDT",
    "MTL/USDT",
    "SNX/USDT",
    "PYR/USDT",
    "DASH/USDT",
    "CITY/USDT",
    "ASTR/USDT",
    "IOTA/USDT",
    "ACM/USDT",
    "BAR/USDT",
    "JUV/USDT",
    "SYS/USDT",
    "RVN/USDT",
    "MBL/USDT",
    "REN/USDT",
    "JST/USDT",
    "OMG/USDT",
    "ATM/USDT",
    "XEC/USDT",
    "STORJ/USDT",
    "ZRX/USDT",
    "SRM/USDT",
    "ICX/USDT",
    "API3/USDT",
    "ONT/USDT",
    "SKL/USDT",
    "MULTI/USDT",
    "QTUM/USDT",
    "COCOS/USDT",
    "VOXEL/USDT",
    "HIVE/USDT",
    "KP3R/USDT",
    "ATA/USDT",
    "STMX/USDT",
    "ADX/USDT",
    "HIGH/USDT",
    "NULS/USDT",
    "MLN/USDT",
    "YGG/USDT",
    "SC/USDT",
    "CKB/USDT",
    "TOMO/USDT",
    "STX/USDT",
    "FLUX/USDT",
    "DNT/USDT",
    "ORN/USDT",
    "PLA/USDT",
    "BADGER/USDT",
    "DF/USDT",
    "MOB/USDT",
    "LPT/USDT",
    "SCRT/USDT",
    "RAD/USDT",
    "NMR/USDT",
    "ELF/USDT",
    "TORN/USDT",
    "T/USDT",
    "QUICK/USDT",
    "LSK/USDT",
    "FIDA/USDT",
    "XNO/USDT",
    "BTG/USDT",
    "GHST/USDT",
    "EPS/USDT"
]

pair_list = Binance_USDT_HALAL
tf = '1m'
oldest_pair = "BTC/USDT"
df_list1m = {}
df_list1d = {}
df_list1h = {}
df_list5m = {}
df_list15m = {}


for pair in pair_list:
    df = get_historical_from_db(ccxt.binance(), pair, '1m', path="./database/")
    df_list1m[pair] = df.loc[:]

for pair in pair_list:
    df = get_historical_from_db(ccxt.binance(), pair, '1d', path="./database/")
    df_list1d[pair] = df.loc[:]

for pair in pair_list:
    df = get_historical_from_db(ccxt.binance(), pair, '1h', path="./database/")
    df_list1h[pair] = df.loc[:]

for pair in pair_list:
    df = get_historical_from_db(ccxt.binance(), pair, '5m', path="./database/")
    df_list5m[pair] = df.loc[:]

for pair in pair_list:
    df = get_historical_from_db(
        ccxt.binance(), pair, '15m', path="./database/")
    df_list15m[pair] = df.loc[:]
del(df)
df_list = df_list1m
prerr("Data load 100% use df_list1d[\"BTC/USDT\"] for exemple to access")


In [5]:
#df_list1d["BTC/USDT"]


In [86]:
# btc1m["high1"]=btc1m.loc[1:]["high"]

def expand_row(dataframe, window=2):
    df = dataframe.copy()
    for i in range(1, window+1):
        df["high"+str(i)] = df["high"][i:]
        df["low"+str(i)] = df["low"][i:]
        df["open"+str(i)] = df["open"][i:]
        df["close"+str(i)] = df["close"][i:]
        df["volume"+str(i)] = df["volume"][i:]
    return df

def expand_previous(dataframe, window=2):
    df = dataframe.copy()
    for i in range(1, window+1):
        try:
            df.loc[window:len(df),"high-"+str(i)]=None
            df["high-"+str(i)].iloc[window:len(df)]=df["high"][window-i:len(df)-i]

            df.loc[window:len(df),"low-"+str(i)]=None
            df["low-"+str(i)].iloc[window:len(df)]=df["low"][window-i:len(df)-i]

            df.loc[window:len(df),"open-"+str(i)]=None
            df["open-"+str(i)].iloc[window:len(df)]=df["open"][window-i:len(df)-i]            
            
            df.loc[window:len(df),"close-"+str(i)]=None
            df["close-"+str(i)].iloc[window:len(df)]=df["close"][window-i:len(df)-i]            
            
            df.loc[window:len(df),"volume-"+str(i)]=None
            df["volume-"+str(i)].iloc[window:len(df)]=df["volume"][window-i:len(df)-i]
            
            # df["high-"+str(i)][i:] = df["high"][i-1:]
            # df["low-"+str(i)][i:] = df["low"][i-1:]
            # df["open-"+str(i)][i:] = df["open"][i-1:]
            # df["close-"+str(i)][i:] = df["close"][i-1:]
            # df["volume-"+str(i)][i:] = df["volume"][i-1:]
        except:
            prerr("Error in     expand_previous: " +str(i))
            
    return df.iloc[window:]

def expand_timeframe(df_minutes,df_hours, window=2):
    dfm = df_minutes.copy()
    for j in range(1, window+1):
        for i in df_hours[dfm.iloc[0].name:].index:
        #prerr(str(i))
            try:
                dfm.loc[pd.date_range(str(i), periods=60, freq="min"),"high_1h-"+str(j)]= df_hours[str(i-pd.Timedelta(str(j)+" hour"))]['high']
                dfm.loc[pd.date_range(str(i), periods=60, freq="min"),"low_1h-"+str(j)]= df_hours[str(i-pd.Timedelta(str(j)+" hour"))]['low']
                dfm.loc[pd.date_range(str(i), periods=60, freq="min"),"open_1h-"+str(j)]= df_hours[str(i-pd.Timedelta(str(j)+" hour"))]['open']
                dfm.loc[pd.date_range(str(i), periods=60, freq="min"),"close_1h-"+str(j)]= df_hours[str(i-pd.Timedelta(str(j)+" hour"))]['close']
            except:
                prerr("Error Merging: "+str(i))
    
    return dfm


def float_or_not(var):
    try:
        x=float(var)
    except:
        x=None
    return x

def expand_to_1h(df_1m,df_1h, window=2):
    dfm = df_1m.copy()
    index_start=df_1h.index.intersection(dfm.index.round(freq='H'))
    for i in index_start:
        for j in range(1, window+1):
            # try:    
                timefragment=dfm.index.intersection(pd.date_range(str(i), periods=60, freq="min"))
                dfm.loc[timefragment,"high_1h-"+str(j)]=float_or_not(df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['high'])
                dfm.loc[timefragment,"low_1h-"+str(j)]=float_or_not(df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['low'])
                dfm.loc[timefragment,"open_1h-"+str(j)]=float_or_not(df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['open'])
                dfm.loc[timefragment,"close_1h-"+str(j)]=float_or_not(df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['close'])
            # except:
            #     prerr("error fonction "str(i))
    return dfm

def expand_to_4h(df_1m,df_4h, window=2):
    dfm = df_1m.copy()
    #index_start=df_1h[str(dfm.iloc[0].name.round(freq='H')):].index.intersection(dfm.index)
    index_start=df_4h.index.intersection(dfm.index.round(freq='4H'))
    for i in index_start:
        for j in range(1, window+1):
            # try:    
                timefragment=dfm.index.intersection(pd.date_range(str(i), periods=4*60, freq="min"))
                dfm.loc[timefragment,"high_4h-"+str(j)]=float_or_not(df_4h.loc[str(i-pd.Timedelta(str(j*4)+" hour"))]['high'])
                dfm.loc[timefragment,"low_4h-"+str(j)]= float_or_not(df_4h.loc[str(i-pd.Timedelta(str(j*4)+" hour"))]['low'])
                dfm.loc[timefragment,"open_4h-"+str(j)]= float_or_not(df_4h.loc[str(i-pd.Timedelta(str(j*4)+" hour"))]['open'])
                dfm.loc[timefragment,"close_4h-"+str(j)]= float_or_not(df_4h.loc[str(i-pd.Timedelta(str(j*4)+" hour"))]['close'])
            # except:
            #     prerr("error fonction "str(i))
    return dfm

def expand_to_1d(df_1m,df_1d, window=2,time_suffix="1d"):
    dfm = df_1m.copy()
    index_start=df_1d.index.intersection(dfm.index.round(freq='1d'))
    for i in index_start:
        for j in range(1, window+1):
            # try:    
                prerr(i)
                timefragment=dfm.index.intersection(pd.date_range(str(i), periods=24*60, freq="min"))
                dfm.loc[timefragment,"high_"+time_suffix+"-"+str(j)]=float_or_not(df_1d.loc[str(i-pd.Timedelta(str(j)+" day"))]['high'])
                dfm.loc[timefragment,"low_"+time_suffix+"-"+str(j)]= float_or_not(df_1d.loc[str(i-pd.Timedelta(str(j)+" day"))]['low'])
                dfm.loc[timefragment,"open_"+time_suffix+"-"+str(j)]= float_or_not(df_1d.loc[str(i-pd.Timedelta(str(j)+" day"))]['open'])
                dfm.loc[timefragment,"close_"+time_suffix+"-"+str(j)]= float_or_not(df_1d.loc[str(i-pd.Timedelta(str(j)+" day"))]['close'])
            # except:
            #     prerr("error fonction "str(i))
    return dfm

def expand_to_5m(df_1m,df_5m, window=2,time_suffix="5m"):
    dfm = df_1m.copy()
    index_start=df_5m.index.intersection(dfm.index.round(freq='5 min'))
    for i in index_start:
        for j in range(1, window+1):
            # try:    
                
                timefragment=dfm.index.intersection(pd.date_range(str(i), periods=5, freq="min"))
                dfm.loc[timefragment,"high_"+time_suffix+"-"+str(j)]=float_or_not(df_5m.loc[str(i-pd.Timedelta(str(j*5)+" min"))]['high'])
                dfm.loc[timefragment,"low_"+time_suffix+"-"+str(j)]= float_or_not(df_5m.loc[str(i-pd.Timedelta(str(j*5)+" min"))]['low'])
                dfm.loc[timefragment,"open_"+time_suffix+"-"+str(j)]= float_or_not(df_5m.loc[str(i-pd.Timedelta(str(j*5)+" min"))]['open'])
                dfm.loc[timefragment,"close_"+time_suffix+"-"+str(j)]= float_or_not(df_5m.loc[str(i-pd.Timedelta(str(j*5)+" min"))]['close'])
            # except:
            #     prerr("error fonction "str(i))
    return dfm


def expand_to_15m(df_1m,df_15m, window=2,time_suffix="15m"):
    dfm = df_1m.copy()
    index_start=df_15m.index.intersection(dfm.index.round(freq='5 min'))
    for i in index_start:
        for j in range(1, window+1):    
            # try:    
                timefragment=dfm.index.intersection(pd.date_range(str(i), periods=15, freq="min"))
                dfm.loc[timefragment,"high_"+time_suffix+"-"+str(j)]=float_or_not(df_15m.loc[str(i-pd.Timedelta(str(j*15)+" min"))]['high'])
                dfm.loc[timefragment,"low_"+time_suffix+"-"+str(j)]= float_or_not(df_15m.loc[str(i-pd.Timedelta(str(j*15)+" min"))]['low'])
                dfm.loc[timefragment,"open_"+time_suffix+"-"+str(j)]= float_or_not(df_15m.loc[str(i-pd.Timedelta(str(j*15)+" min"))]['open'])
                dfm.loc[timefragment,"close_"+time_suffix+"-"+str(j)]= float_or_not(df_15m.loc[str(i-pd.Timedelta(str(j*15)+" min"))]['close'])
            # except:
            #     prerr("error fonction "str(i))
    return dfm

def rapid1d_expand(df1m,df1d,window=2):
    d1min=df1m.copy()
    d1day=df1d.loc[
    d1min.index[0].round(freq='1d'):
    d1min.index[len(d1min)-1].round(freq='1d')+pd.Timedelta('1 day')
    ].copy()
    d1day_pre=expand_previous(d1day,window)
    d1day_pre=d1day_pre.drop(columns=['open', 'low','close','high','volume'])
    d1day_pre=d1day_pre.add_suffix("_day")
    d1min=pd.merge_asof(
    d1min, d1day_pre, on=None, left_on=None, right_on=None, left_index=True, 
    right_index=True, by=None, left_by=None, right_by=None, 
    suffixes=('', '_day'),
    tolerance=pd.Timedelta('1 day'), allow_exact_matches=True, direction='backward')
    return d1min

def rapid1h_expand(df1m,df1h,window=2):
    d1min=df1m.copy()
    d1hour=df1h.loc[
    d1min.index[0].round(freq='H'):
    d1min.index[len(d1min)-1].round(freq='H')+pd.Timedelta('1 hour')
    ].copy()
    d1hour_pre=expand_previous(d1hour,window)
    d1hour_pre=d1hour_pre.drop(columns=['open', 'low','close','high','volume'])
    d1hour_pre=d1hour_pre.add_suffix("_hour")
    d1min=pd.merge_asof(
    d1min, d1hour_pre, on=None, left_on=None, right_on=None, left_index=True, 
    right_index=True, by=None, left_by=None, right_by=None, 
    suffixes=('', '_hour'),
    tolerance=pd.Timedelta('1 hour'), allow_exact_matches=True, direction='backward')
    return d1min


def rapid5m_expand(df1m,df5m,window=2):
    d1min=df1m.copy()
    d5min=df5m.loc[
    d1min.index[0].round(freq='5 min'):
    d1min.index[len(d1min)-1].round(freq='5 min')+pd.Timedelta('5 min')
    ].copy()
    d5min_pre=expand_previous(d5min,window)
    d5min_pre=d5min_pre.drop(columns=['open', 'low','close','high','volume'])
    d5min_pre=d5min_pre.add_suffix("_5min")
    d1min=pd.merge_asof(
    d1min, d5min_pre, on=None, left_on=None, right_on=None, left_index=True, 
    right_index=True, by=None, left_by=None, right_by=None, 
    suffixes=('', '_5min'),
    tolerance=pd.Timedelta('5 min'), allow_exact_matches=True, direction='backward')
    return d1min

def rapid15m_expand(df1m,df15m,window=2):
    d1min=df1m.copy()
    d15min=df15m.loc[
    d1min.index[0].round(freq='15 min'):
    d1min.index[len(d1min)-1].round(freq='15 min')+pd.Timedelta('15 min')
    ].copy()
    d15min_pre=expand_previous(d15min,window)
    d15min_pre=d15min_pre.drop(columns=['open', 'low','close','high','volume'])
    d15min_pre=d15min_pre.add_suffix("_15min")
    d1min=pd.merge_asof(
    d1min, d15min_pre, on=None, left_on=None, right_on=None, left_index=True, 
    right_index=True, by=None, left_by=None, right_by=None, 
    suffixes=('', '_15min'),
    tolerance=pd.Timedelta('15 min'), allow_exact_matches=True, direction='backward')
    return d1min

def full_expand(df1m,df5m,df15m,df1h,df1d,window=2):
    d1min=df1m.copy()
    d1min=expand_previous(d1min)
    d1min=rapid1d_expand(d1min,df1d,window)
    d1min=rapid1h_expand(d1min,df1h,window)
    d1min=rapid15m_expand(d1min,df15m,window)
    d1min=rapid5m_expand(d1min,df5m,window)
    return d1min

def day_expand(data_full):
    ser = pd.to_datetime(pd.Series(data_full.index))
    data_full["day"]=ser.dt.isocalendar().day.values
    data_full["hour"]=ser.dt.hour.values
    data_full["minute"]=ser.dt.minute.values

def Meta_expand(data_full,metadt,pair):
    data_full["lunch_day"]=metadt["" pair]


# merging
def pair_btc(pair="LTC/USDT",window=2):
    Pair_Full=full_expand(df_list1m[pair],df_list5m[pair],df_list15m[pair],df_list1h[pair],df_list1d[pair],window)
    BTC_Full=full_expand(
        df_list1m["BTC/USDT"].loc[df_list1m[pair].iloc[0].name:
        df_list1m[pair].iloc[len(df_list1m[pair])-1].name],
        df_list5m["BTC/USDT"],df_list15m["BTC/USDT"],df_list1h["BTC/USDT"],df_list1d["BTC/USDT"],window)   
    BTC_Full=BTC_Full.add_prefix("BTC_")
    Merged=pd.merge(Pair_Full, BTC_Full, left_index=True, how='outer',
            right_index=True, suffixes=('', ''))
    day_expand(Merged)
    return Merged

def mini_expand(pair="LTC/USDT",i=0,j=10000,window=2):
    Pair_Full=full_expand(df_list1m[pair].iloc[i:j],df_list5m[pair],df_list15m[pair],df_list1h[pair],df_list1d[pair],window)
    BTC_Full=full_expand(
        df_list1m["BTC/USDT"].loc[Pair_Full.iloc[0].name-pd.Timedelta(str(window-1) +" min"):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list5m["BTC/USDT"],
        df_list15m["BTC/USDT"],
        df_list1h["BTC/USDT"],
        df_list1d["BTC/USDT"],
        window)   
    BTC_Full=BTC_Full.add_prefix("BTC_")
    Merged=pd.merge(Pair_Full, BTC_Full, left_index=True, how='left',
            right_index=True, suffixes=('', ''))
    day_expand(Merged)
    return Merged




In [85]:
Merged.loc["2020-11-16 21:19:00"]["BTC_high-1"]==df_list1m["BTC/USDT"].loc["2020-11-16 21:18:00"]["high"]

True

In [7]:
# def expand_to_1h(df_1m,df_1h, window=2):
#     dfm = df_1m.copy()
#     #index_start=df_1h[str(dfm.iloc[0].name.round(freq='H')):].index.intersection(dfm.index)
#     index_start=df_1h.index.intersection(dfm.index.round(freq='H'))
#     for j in range(1, window+1):
#         for i in index_start:
#             # try:    
#                 timefragment=dfm.index.intersection(pd.date_range(str(i), periods=60, freq="min"))
#                 #prerr(df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['high'].values[0])
#                 dfm.loc[timefragment,"high_1h-"+str(j)]=df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['high'].values[0]
#                 dfm.loc[timefragment,"low_1h-"+str(j)]= df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['low'].values[0]
#                 dfm.loc[timefragment,"open_1h-"+str(j)]= df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['open'].values[0]
#                 dfm.loc[timefragment,"close_1h-"+str(j)]= df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['close'].values[0]
#             # except:
#             #     prerr("error fonction "str(i))
#     return dfm


In [ ]:
window=3
pair="LTC/USDT"
i=0
j=100000
def slow_expand(pair="LTC/USDT",i=0,j=100000,window=3):
    df=mini_expand(pair=pair,i=i,j=j,window=window)
    for mx in range(1,int(len(df_list1m[pair])/j)+1) :
        df=pd.concat([df,
        mini_expand(pair=pair,
        i=(mx*j)-window,
        j=(mx+1)*j,
        window=window)],axis=0)
        print("loop "+str(mx)+"--> size of df: "+str(len(df)))

#    print("loop "+str(mx)+"--> size of df: "+str(len(df)))

In [123]:
testdate=df.iloc[-555000].name
print(testdate)
df.loc[testdate]["BTC_open-3_5min"]

2021-04-28 03:39:00


55186.88

In [ ]:
#btc1m = expand_previous(df_list1m["BTC/USDT"].iloc[0:10000], 3)
#df=pair_btc(pair="LTC/USDT",window=30)
df=slow_expand(pair="LTC/USDT",i=0,j=100000,window=120)

In [124]:
df[df.isnull().any(axis=1)]


,open,high,low,close,volume,high-1,low-1,open-1,close-1,volume-1,...,BTC_close-2_5min,BTC_volume-2_5min,BTC_high-3_5min,BTC_low-3_5min,BTC_open-3_5min,BTC_close-3_5min,BTC_volume-3_5min,day,hour,minute
date,,,,,,,,,,,,,,,,,,,,,
2020-07-01 00:02:00,41.07,41.07,40.96,41.00,952.62714,41.15,41.07,41.14,41.07,264.70116,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0,2
2020-07-01 00:03:00,40.98,41.04,40.98,41.02,19.45277,41.07,40.96,41.07,41.00,952.62714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0,3
2020-07-01 00:04:00,41.02,41.04,41.02,41.04,111.24215,41.04,40.98,40.98,41.02,19.45277,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0,4
2020-07-01 00:05:00,41.04,41.04,40.99,40.99,478.42388,41.04,41.02,41.02,41.04,111.24215,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0,5
2020-07-01 00:06:00,40.99,41.03,40.97,41.03,52.08284,41.04,40.99,41.04,40.99,478.42388,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-18 19:59:00,68.50,68.50,68.30,68.50,483.86100,68.50,68.30,68.40,68.50,778.81300,...,29312.15,277.84432,29397.53,29269.92,29321.73,29272.20,222.33866,3,19,59
2022-05-18 20:00:00,68.40,68.50,68.30,68.30,447.32600,68.50,68.30,68.50,68.50,483.86100,...,29314.11,278.74192,29336.20,29266.62,29272.20,29312.15,277.84432,3,20,0
2022-05-18 20:01:00,68.30,68.40,68.30,68.40,13.85900,68.50,68.30,68.40,68.30,447.32600,...,29314.11,278.74192,29336.20,29266.62,29272.20,29312.15,277.84432,3,20,1


In [60]:
#ltc1m = expand_row(df_list1m["LTC/USDT"], 3)
sys.getsizeof(df)/(1024**2)

5914.110889434814

In [59]:
# df.to_excel("D:\+DATA+\LTCtest.xlsx", sheet_name='LTC', na_rep='', float_format=None,
#  columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, encoding=None, 
# inf_rep='inf', verbose=True, freeze_panes=None, storage_options=None)
df.to_csv("D:\+DATA+\LTCtest.csv")

In [125]:
b=ccxt.binance()
b.set_sandbox_mode(enabled=True)

In [ ]:
b.api

In [127]:
'''
Binance First Candle Finders
Creslin

Get list of all IDs on binance
Returns the first candle / launch timestamp to the minute for each
'''
import urllib.request
import json
import ccxt

def all_ids():
    # load all markets from binance into a list
    id = 'binance'
    exchange_found = id in ccxt.exchanges
    if exchange_found:
        exchange = getattr(ccxt, id)({})
        markets = exchange.load_markets()
        tuples = list(ccxt.Exchange.keysort(markets).items())

        ids = []
        for (k, v) in tuples:
            ids.append(v['id'])

        return ids

def give_first_kline_open_stamp(interval, symbol, start_ts=1499990400000):
        '''
        Returns the first kline from an interval and start timestamp and symbol
        :param interval:  1w, 1d, 1m etc - the bar length to query
        :param symbol:    BTCUSDT or LTCBTC etc
        :param start_ts:  Timestamp in miliseconds to start the query from
        :return:          The first open candle timestamp
        '''

        url_stub = "http://api.binance.com/api/v1/klines?interval="

        #/api/v1/klines?interval=1m&startTime=1536349500000&symbol=ETCBNB
        addInterval   = url_stub     + str(interval) + "&"
        addStarttime  = addInterval   + "startTime="  + str(start_ts) + "&"
        addSymbol     = addStarttime + "symbol="     + str(symbol)
        url_to_get = addSymbol

        kline_data = urllib.request.urlopen(url_to_get).read().decode("utf-8")
        kline_data = json.loads(kline_data)

        return kline_data[0][0]


# Get list of all IDs on binance
ids = []
#ids = all_ids()
for halalpair in Binance_USDT_HALAL:
#    print( halalpair.replace('/',''))
    ids.append(halalpair.replace('/',''))
#print(ids)
MetaData=pd.DataFrame(ids)
MetaData["Pair"]=Binance_USDT_HALAL
counters=0
for this_id in ids:
    '''
    Find launch Week of symbol, start at Binance launch date 2017-07-14 (1499990400000)
    Find launch Day of symbol in week
    Find launch minute of symbol in day
    '''

    symbol_launch_week_stamp   = give_first_kline_open_stamp('1w', this_id, 1499990400000 )
    symbol_launch_day_stamp    = give_first_kline_open_stamp('1d', this_id, symbol_launch_week_stamp)
    symbol_launch_minute_stamp = give_first_kline_open_stamp('1m', this_id, symbol_launch_day_stamp)
    MetaData.loc[counters,"launch_week_stamp"]=str(symbol_launch_week_stamp)
    MetaData.loc[counters,"launch_day_stamp"]=str(symbol_launch_day_stamp)
    MetaData.loc[counters,"launch_minute"]=pd.to_datetime(symbol_launch_minute_stamp, unit='ms')

    counters += 1

    #print("Week stamp", symbol_launch_week_stamp)
    #print("Day  stamp", symbol_launch_day_stamp)
    #print("Min  stamp", symbol_launch_minute_stamp)

    print(this_id, "launched", symbol_launch_minute_stamp )
    #print("")

BTCUSDT launched 1502942400000
LUNAUSDT launched 1598004000000
ETHUSDT launched 1502942400000
GMTUSDT launched 1646827200000
USTUSDT launched 1640332800000
SOLUSDT launched 1597125600000
APEUSDT launched 1647527400000
XRPUSDT launched 1525421460000
IDEXUSDT launched 1631174400000
AVAXUSDT launched 1600756200000
DOTUSDT launched 1597791600000


ADAUSDT launched 1523937720000
JASMYUSDT launched 1637582400000
TRXUSDT launched 1528716600000
NEARUSDT launched 1602651600000
AXSUSDT launched 1604494800000
GALUSDT launched 1651752000000
GALAUSDT launched 1631512800000
SHIBUSDT launched 1620644400000


ZILUSDT launched 1550574240000


ENSUSDT launched 1636527600000
DOGEUSDT launched 1562328000000


LTCUSDT launched 1513135920000
EURUSDT launched 1578038400000
MANAUSDT launched 1596708000000
DARUSDT launched 1636012800000


WAVESUSDT launched 1547805600000
LAZIOUSDT launched 1634817600000
ALICEUSDT launched 1615788000000


ROSEUSDT launched 1605772800000
ZECUSDT launched 1553140800000
ALGOUSDT launched 1561161600000
GRTUSDT launched 1608237000000
PSGUSDT launched 1608530400000
SLPUSDT launched 1619784000000
EOSUSDT launched 1527483600000
PORTOUSDT launched 1637064000000
ICPUSDT launched 1620694800000
EGLDUSDT launched 1599102000000
XMRUSDT launched 1552622400000
KDAUSDT launched 1646998200000
ETCUSDT launched 1528770600000
MBOXUSDT launched 1629360000000
OGNUSDT launched 1578538800000
ARUSDT launched 1620993600000
GLMRUSDT launched 1641916800000
LOKAUSDT launched 1642680000000
XLMUSDT launched 1527759000000
MTLUSDT launched 1566554400000
SNXUSDT launched 1594296000000
PYRUSDT launched 1637913600000
DASHUSDT launched 1553745600000
CITYUSDT launched 1636531200000
ASTRUSDT launched 1646047800000
IOTAUSDT launched 1527759000000
ACMUSDT launched 1614164400000
BARUSDT launched 1619002800000
JUVUSDT launched 1608530400000
SYSUSDT launched 1632477600000
RVNUSDT launched 1569412800000
MBLUSDT launched 15822936000

In [128]:
MetaData

,0,Pair,launch_week_stamp,launch_day_stamp,launch_minute
0,BTCUSDT,BTC/USDT,1502668800000,1502928000000,2017-08-17 04:00:00
1,LUNAUSDT,LUNA/USDT,1597622400000,1597968000000,2020-08-21 10:00:00
2,ETHUSDT,ETH/USDT,1502668800000,1502928000000,2017-08-17 04:00:00
3,GMTUSDT,GMT/USDT,1646611200000,1646784000000,2022-03-09 12:00:00
4,USTUSDT,UST/USDT,1639958400000,1640304000000,2021-12-24 08:00:00
...,...,...,...,...,...
106,FIDAUSDT,FIDA/USDT,1632700800000,1632960000000,2021-09-30 12:00:00
107,XNOUSDT,XNO/USDT,1642982400000,1643328000000,2022-01-28 08:00:00
108,BTGUSDT,BTG/USDT,1618185600000,1618531200000,2021-04-16 07:00:00
109,GHSTUSDT,GHST/USDT,1629072000000,1629417600000,2021-08-20 10:00:00
